Define the Table List

In [0]:
# List of tables and their primary key columns
tables = [
    {"name": "media_customer_reviews", "key": "franchiseID", "path": "/FileStore/tables/media_customer_reviews.parquet"},
    {"name": "media_gold_reviews_chunked", "key": "franchiseID", "path": "/FileStore/tables/media_gold_reviews_chunked.parquet"},
    {"name": "sales_customers", "key": "customerID", "path": "/FileStore/tables/sales_customers.parquet"},
    {"name": "sales_franchises", "key": "franchiseID", "path": "/FileStore/tables/sales_franchises.parquet"},
    {"name": "sales_suppliers", "key": "supplierID", "path": "/FileStore/tables/sales_suppliers.parquet"},
    {"name": "sales_transactions", "key": "transactionID", "path": "/FileStore/tables/sales_transactions.parquet"}
]

Create Schemas - Bronze, Silver & Gold

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

Deduplication Function

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col
 
def deduplicate_by_key(df, key, order_col=None):
    if order_col and order_col in df.columns:
        window_spec = Window.partitionBy(key).orderBy(col(order_col).desc())
    else:
        window_spec = Window.partitionBy(key).orderBy(key)
    return df.withColumn("rn", row_number().over(window_spec)).filter("rn = 1").drop("rn")

Dynamic Bronze -> Silver -> Gold ETL Function

In [0]:
def run_etl_pipeline(table_name: str, key: str, file_path: str):
    bronze = f"bronze.{table_name}"
    silver = f"silver.{table_name}"
    
    # Load raw data
    df = spark.read.parquet(file_path)
    
    # Write Bronze table
    df.write.format("delta").mode("overwrite").saveAsTable(bronze)
    
    # Deduplicate before merge
    df_dedup = deduplicate_by_key(df, key, order_col="last_updated" if "last_updated" in df.columns else None)
    
    # Create Silver table if it doesn't exist
    if not spark.catalog.tableExists(silver):
        df_dedup.write.format("delta").mode("overwrite").saveAsTable(silver)
        return
    
    # Get dynamic column list
    columns = df.columns
    update_expr = ", ".join([f"target.{c} = source.{c}" for c in columns])
    insert_cols = ", ".join(columns)
    insert_vals = ", ".join([f"source.{c}" for c in columns])
    
    # Perform SCD Type 1 Merge
    merge_sql = f"""
    MERGE INTO {silver} AS target
    USING (SELECT * FROM bronze.{table_name}) AS source
    ON target.{key} = source.{key}
    WHEN MATCHED THEN UPDATE SET {update_expr}
    WHEN NOT MATCHED THEN INSERT ({insert_cols}) VALUES ({insert_vals})
    """
    spark.sql(merge_sql)

Loop through All Tables

In [0]:
for t in tables:
    run_etl_pipeline(t["name"], t["key"], t["path"])

Query the Gold Layer

In [0]:
%sql
-- Get the most sold products to identify the top-selling items.

CREATE OR REPLACE TABLE gold.top_selling_products AS
SELECT
    product,
    SUM(quantity) AS total_quantity_sold
FROM silver.sales_transactions
GROUP BY product
ORDER BY total_quantity_sold DESC;

select * from gold.top_selling_products;

product,total_quantity_sold
Golden Gate Ginger,3865
Outback Oatmeal,3733
Austin Almond Biscotti,3716
Tokyo Tidbits,3662
Pearly Pies,3595
Orchard Oasis,3586


In [0]:
%sql
-- Find which suppliers provide ingredients to the most franchises.

CREATE OR REPLACE TABLE gold.most_suppliers AS
SELECT
    sup.name,
    COUNT(DISTINCT fs.franchiseID) AS total_franchises
FROM silver.sales_franchises fs
JOIN silver.sales_suppliers sup ON fs.supplierID = sup.supplierID
GROUP BY sup.name
ORDER BY total_franchises DESC;

select * from gold.most_suppliers;

name,total_franchises
Coconut Grove,1
Maple Monarch,1
Sesame Seeds,1
Raisin Ranch,1
Ginger Gems,1
Cacao Wonders,1
Cinnamon Spice,1
Anise Acres,1
Honey Hives,1
Fennel Fields,1


In [0]:
%sql
-- Get total sales per month.

CREATE OR REPLACE TABLE gold.total_sales_per_month AS
SELECT
    DATE_FORMAT(dateTime, 'yyyy-MM') AS sale_month,
    SUM(totalPrice) AS total_sales
FROM silver.sales_transactions
GROUP BY sale_month
ORDER BY sale_month;

select * from gold.total_sales_per_month;

sale_month,total_sales
2024-05,66471
